In [ ]:
%load_ext autoreload
%autoreload 2

## Dataset 준비


CIFAR-10 dataset은 10개의 카테고리로 구성 되어 있으며 각 카테고리당 5000개의 이미지로 총 50000 이미지로 구성되어 있습니다.

Amazon image classification algorithm은 두 종류의 input format을 지원 합니다.

1. [recordio](https://mxnet.incubator.apache.org/tutorials/basic/record_io.html) 
2. [lst - im2rec](https://mxnet.incubator.apache.org/how_to/recordio.html?highlight=im2rec)

CIFAR-10두 종류의 파일은 http://data.mxnet.io/data/cifar10/ 에서 다운 받으실 수 있습니다.

>본 가이드에서는 미리 준비된 CIFAR-10 recordio 파일을 사용하였습니다.

## 이전 작업에서 생성한 S3 Bucket 이름을 `bucket` 변수에 입력하시기 바랍니다.

- `bucket`

In [ ]:
bucket='YOUR_BUCKET_NAME'

## 유틸 함수

- `download` CIFAR-10 dataset 다운로드
- `upload_to_s3` CIFAR-10-dataset을 s3로 업로드
- `list_s3` 업로드한 dataset 확인

In [ ]:
import os
from six.moves import urllib
import boto3

#download utility function
#다운로드 유틸 함수
def download(url,path="/tmp"):
    filename = url.split("/")[-1]
    filepath = os.path.join(path,filename)
    if not os.path.exists(filepath):
        urllib.request.urlretrieve(url, filepath)
    else:
        print("{} exsits!".format(filepath))

#S3 upload utility function
#S3 업로드 유틸 함수
def upload_to_s3(channel, filename, bucket, path="/tmp"):
    s3 = boto3.resource('s3')
    filepath = os.path.join(path,filename)
    data = open(filepath, "rb")
    key = channel + '/' + filename
    s3.Bucket(bucket).put_object(Key=key, Body=data)

#S3 upload check util function
#S3 업로드 확인 유틸 함수
def list_s3(bucket):
    s3 = boto3.resource('s3')
    for file in s3.Bucket(bucket).objects.all():
        print(file.key)

In [ ]:
%%time
## Download
## 다운로드 CIFAR10 dataset
download('http://data.mxnet.io/data/cifar10/cifar10_train.rec')
download('http://data.mxnet.io/data/cifar10/cifar10_val.rec')

In [ ]:
%%time
## Upload
## S3 업로드 CIFAR10 dataset
upload_to_s3('validation', 'cifar10_val.rec',bucket)
upload_to_s3('train', 'cifar10_train.rec',bucket)

In [ ]:
## Check
## S3 업로드 확인
list_s3(bucket)